## Produkce

#### Jabloně – plocha sadů podle staří – ČR (ČSÚ)

In [ ]:
import pandas as pd

# df = pd.read_csv("data/jablone_plocha_sadu_stari_2012_CSU.csv", encoding='utf-8')
# df

variety_files = ["data/jablone_plocha_sadu_stari_2012_CSU.csv", "data/jablone_plocha_sadu_stari_2017_CSU.csv"]
for file in variety_files:
    # Turn each file into a pd df
    df = pd.read_csv(file, encoding='utf-8')

    # Drop junk rows/columns        
    df = df.drop(range(0,4))
    df = df.drop(range(19,24))
    df.drop(df.columns[0], axis=1, inplace=True)
#     df.drop(df.columns[0], axis=1, inplace=True)
    df.iloc[2, 0] = 'CELKEM'
    df = df.fillna('EMPTY')

    # Save clean csv
    df.to_csv(f"data_clean/{file[4:]}", index=False, header=False, encoding='utf-8')

#### Jabloně – plocha sadů podle hustoty – ČR (ČSÚ)

In [ ]:
import pandas as pd

area_files = ["data/jablone_plocha_sadu_hustota_2012_CSU.csv", "data/jablone_plocha_sadu_hustota_2017_CSU.csv"]

for file in area_files:
    # Turn each file into a pd df
    df = pd.read_csv(file, encoding='utf-8')

    # Drop junk rows/columns        
    df = df.drop(range(0,5))
    df = df.drop(range(22,26))
    df.drop(df.columns[0], axis=1, inplace=True)
    df.drop(df.columns[0], axis=1, inplace=True)
    df.iloc[2, 1] = 'CELKEM'
    df = df.fillna('EMPTY')

    # Save clean csv
    df.to_csv(f"data_clean/{file[4:]}", index=False, header=False, encoding='utf-8')

#### Plocha sadů – EU (Eurostat)

In [ ]:
import pandas as pd

df = pd.read_csv("data/orch_total_linear.csv", sep=';')
df = df.drop(df.columns[[0, 1, 2, 3, 4, 5, -1]], axis=1)
df.describe()

# # Make 4 tables by years
df_2002 = df[df['TIME_PERIOD'] == 2002] #55
df_2007 = df[df['TIME_PERIOD'] == 2007] #118
df_2012 = df[df['TIME_PERIOD'] == 2012] #330
df_2017 = df[df['TIME_PERIOD'] == 2017] #154


# Merge them into one by country and fruitvar
df1 = pd.merge(df_2002, df_2007, on=['fruitvar', 'geo'], suffixes=('_2002', '_2007'), how='right')
df2 = pd.merge(df1, df_2012, on=['fruitvar', 'geo'], how='right')
df3 = pd.merge(df2, df_2017, on=['fruitvar', 'geo'], suffixes=('_2012', '_2017'), how='left')
df = df3.drop(df3.columns[[2, 4, 6, 8]], axis=1)
df = df[['geo', 'fruitvar', 'OBS_VALUE_2002', 'OBS_VALUE_2007', 'OBS_VALUE_2012', 'OBS_VALUE_2017']]
df = df.rename(columns={'geo': 'COUNTRY', 'fruitvar': 'FRUIT'})
df

# Save clean csv
df.to_csv(f"data_clean/plocha_sadu_ha_eu_2002_2017.csv", index=False, encoding="utf-8")

# clean = pd.read_csv("data_clean/plocha_sadu_ha_eu_2002_2017.csv", encoding='utf-8')
# clean

#### Porovnání údajů o sklizních sadů a domácností: ČSÚ vs. OU ČR (Situační zpráva MZ)

In [ ]:
import pandas as pd


# Read data from the file
with open("data/sady_str_skilzni.txt", encoding='utf-8-sig') as f:
    data = f.readlines()

# Clean up junk chars
data = [line.strip('\n') for line in data]
data = [line for line in data if line != '']

# Break the list up into lists starting with individual years
lists_by_year = []
current_sublist = []

for i in data:
    if i.isdigit():
        # If current_sublist already exists, add more data
        if current_sublist:
            lists_by_year.append(current_sublist)
        # If it doesn't create one starting with the year
        current_sublist = [i]
    else:
        current_sublist.append(i)

if current_sublist:
    lists_by_year.append(current_sublist)

# Make a list of dicts
dicts_by_year = []

years = range(0, 12)
for y in years:
    # Create subdicts for each year
    years_dict = {}
    for i in lists_by_year[y]:
        # Create a KV pair for year
        if i.isdigit():
            years_dict['rok'] = i
        # And KV pairs for other data
        else:
            i_split = i.split()
            k = ' '.join(i_split[:2])
            v = ''.join(i_split[2:])
            years_dict[k] = v
    
    dicts_by_year.append(years_dict)

# Write to csv
df = pd.DataFrame(dicts_by_year)
df.to_csv(f"data_clean/sklizne_ou.csv", index=False, encoding="utf-8")
# clean = pd.read_csv("data_clean/sklizne_ou.csv", encoding='utf-8')
# clean

## Spotřeba

 #### Spotřeba ovoce ČR 1948-2022 (ČSÚ)

In [ ]:
import pandas as pd


df = pd.read_excel("data/spotreba_ovoce_cr_CSU.xlsx", header=None)

df = df.drop([0, 1, 2, 3, 4, 5, 6, 8, 9])

df = df.drop(df.columns[[0, -1]], axis=1)

df.iloc[0, 0] = 'rok'
df.iloc[0, 1] = 'ovoce_total'
df.iloc[0, 2] = 'ovoce mírného pásma'
df.iloc[0, 16] = 'jižní ovoce'
df.iloc[0, 21] = 'ananas'
df.iloc[0, 22] = 'kiwi'

df = df.replace('\n', ' ', regex=True)
df = df.replace('.', 'Unknown')
df

df.to_csv(f"data_clean/spotreba_ovoce_cr_CSU.csv", index=False, header=False, encoding="utf-8")
# clean = pd.read_csv("data_clean/spotreba_ovoce_cr_CSU.csv", header=None, encoding='utf-8')
# clean


## Misc

#### Odrudy jablek (ČSÚ)

In [ ]:
import pandas as pd


df = pd.read_excel("data/seznam_odrud.xlsx")
df = df.drop(df.columns[range(6,19)], axis=1)
df = df.drop(range(281, 304))

df.to_csv(f"data_clean/seznam_odrud.csv", index=False, encoding="utf-8")
# clean = pd.read_csv("data_clean/seznam_odrud.csv", encoding='utf-8')
# clean.tail(30)


#### Pamatné stromy (ČSÚ)

In [ ]:
import pandas as pd


df = pd.read_csv("data/pamatne_stromy.csv", sep=';', encoding='utf-8')
df = df.drop(df.columns[1], axis=1)

df.to_csv(f"data_clean/pamatne_stromy.csv", index=False, encoding="utf-8")
# clean = pd.read_csv("data_clean/pamatne_stromy.csv", encoding='utf-8')
# clean

#### Crop production EU 1955-1999 (Eurostat)

In [ ]:
import pandas as pd


df = pd.read_csv("data/crop_prod_historical_1955_1999.csv", encoding='utf-8')
df = df.drop(df.columns[[0, 1, 2, 8]], axis=1)
df = df.rename(columns={'geo': 'COUNTRY', 'strucpro': 'STR_OF_PRODUCTION', 'crops': 'CROPS', 'TIME_PERIOD': 'YEAR'})

labels = {'F0000': 'Fruits, berries and nuts', 'F1110': 'Apples', 'F1111': 'Apples for fresh consumption', 'F1112': 'Apples for processing'}
df["CROPS"] =  df["CROPS"].map(labels)

# df = df.replace('F0000', 'Fruits, berries and nuts')
# df = df.replace('F1110', 'Apples')
# df = df.replace('F1111', 'Apples for fresh consumption')
# df = df.replace('F1111', 'Apples for fresh consumption')

df.to_csv(f"data_clean/crop_prod_historical_1955_1999.csv", index=False, encoding="utf-8")
# clean = pd.read_csv("data_clean/crop_prod_historical_1955_1999.csv", encoding='utf-8')
# clean

#### Počet stromů (ČSÚ)

In [ ]:
import pandas as pd


df = pd.read_excel("data/pocet_stromu_sklizen_2002_2023.xlsx", header=None)
# df = df.drop(range(0, 5))
# df = df.drop(range(31))
df = df.iloc[range(5,31)]
df = df.drop(df.columns[[0, 1]], axis=1)
df = df.rename(columns={'2002': 'Dummy'})

# df = df.replace('\n', ' ', regex=True)
# df = df.replace(',', '')

df = df.transpose()

df.to_csv(f"data_clean/ovocne_stromy_kere_2002_2023_tr.csv", index=False, header=False, encoding="utf-8")
# clean = pd.read_csv("data_clean/ovocne_stromy_kere_2002_2023_tr.csv", encoding='utf-8')
# clean


#### Sklizeň (ČSÚ)

In [ ]:
import pandas as pd


df = pd.read_excel("data/pocet_stromu_sklizen_2002_2023.xlsx", header=None)
df = df.drop(range(0, 5))
df = df.drop(range(6, 31))
df = df.drop(range(56, 85))
df = df.drop(df.columns[[0, 1]], axis=1)

df = df.transpose()

df.to_csv(f"data_clean/sklizen_2002_2023_tr.csv", index=False, header=False, encoding="utf-8")
clean = pd.read_csv("data_clean/sklizen_2002_2023_tr.csv", encoding='utf-8')
clean

#### Produkční plocha (ČSÚ)

In [ ]:
import pandas as pd


df = pd.read_excel("data/pocet_stromu_sklizen_2002_2023.xlsx", header=None)
df = df.drop(range(0, 5))
df = df.drop(range(6, 56))
df = df.drop(range(81, 85))
df = df.drop(df.columns[[0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11]], axis=1)

df = df.transpose()

df.to_csv(f"data_clean/prod_plocha_2011_2023_tr.csv", index=False, header=False, encoding="utf-8")
# clean = pd.read_csv("data_clean/prod_plocha_2011_2023_tr.csv", encoding='utf-8')
# clean

## One more round of cleaning

#### Cleaning up tricky commas for keboola uplaod

In [36]:
for year in range(2011, 2024):
    with open(f"data/import_export_{year}.csv", 'r', encoding='ISO-8859-1') as file:
        lines = file.readlines()
    
    # Remove commas and write output files to data_clean
    with open(f"data_clean/import_export_{year}.csv", 'w', encoding='utf-8') as new_file:
        for line in lines:
            if line.endswith(",\n"):
                new_file.write(line[:-2] + "\n")
            else:
                new_file.write(line)


#### Transposing tables to join on years

In [ ]:
import pandas as pd


df = pd.read_csv("data/sklizen_2002_2023.csv", encoding='utf-8')
df = df.transpose()

df.to_csv(f"data_clean/sklizen_2002_2023_tr.csv", header=False, encoding="utf-8")
# clean = pd.read_csv("data_clean/sklizen_2002_2023_tr.csv", encoding='utf-8')
# clean